### Step 1. Importing and preprocessing the dataset

In [4]:
import os # helps to read images froma a directory
import pandas as pd 
import numpy as np 
from tensorflow.keras.preprocessing.image import ImageDataGenerator # loading and processing images for Deep Learning
from tensorflow.keras.models import Sequential # to build a Deep Learning model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # layers for the Deep Learning model
from tensorflow.keras.optimizers import Adam # optimizer for training the model
from sklearn.metrics import classification_report, confusion_matrix # evaluating the model

In [5]:
train_dir = 'data/chest_xray/train'
categories = ['NORMAL', 'PNEUMONIA']
filepaths = []
labels = []

In [6]:
# Loop through each category and get file paths and labels
for category in categories:
    folder = os.path.join(train_dir, category)
    for fname in os.listdir(folder):
        filepaths.append(f"{category}/{fname}")
        labels.append(category)

In [7]:
df = pd.DataFrame({'Filename': filepaths, 'Label': labels})

In [ ]:
# rescaling pixel values (lower it) and setting aside validation data (80% training and 20% validation)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [9]:
# Loading training data from dataframe
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_dir,
    x_col='Filename',
    y_col='Label',
    subset='training',
    batch_size=32, # processing 32 images at a time
    seed=42, # starting point for random operations
    shuffle=True, # shuffling data before the training
    class_mode='binary', # binary classification (NORMAL vs PNEUMONIA)
    target_size=(150, 150) # resizing images to 150x150 pixels
)

Found 4173 validated image filenames belonging to 2 classes.


In [10]:
# validation data from dataframe
valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_dir,
    x_col='Filename',
    y_col='Label',
    subset='validation',
    batch_size=32, # processing 32 images at a time
    seed=42, # starting point for random operations
    shuffle=True, # shuffling data before the training
    class_mode='binary', # binary classification (NORMAL vs PNEUMONIA)
    target_size=(150, 150) # resizing images to 150x150 pixels
)

Found 1043 validated image filenames belonging to 2 classes.


### Step 2. Building a CNN model (Convolution, ReLU, Pooling, Dense Layers)

In [11]:
''' Building the CNN model
    Sequential model adds one layer after another in sequence, like a pipeline. Each layer transforms the data in some way.
    Conv2D layer applies convolution operation to extract features from images using filters/kernels.
    MaxPooling2D layer reduces the spatial dimensions (width and height) of the feature maps, retaining important information while reducing computational load.
    Flatten layer converts the 2D feature maps into a 1D vector, preparing it for the fully connected layers.
    Dropout layer randomly sets a fraction of input units to 0 during training to prevent overfitting.
    Dense layer is a fully connected layer where each neuron is connected to every neuron in the previous layer.
    The final Dense layer with a sigmoid activation function outputs a probability score between 0 and 1 for binary classification (NORMAL vs PNEUMONIA).
'''

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)), # finding the patterns in the images; 3 channels for RGB.
    MaxPooling2D((2, 2)), 
    Conv2D(64, (3, 3), activation='relu'), # 64 filters to learn more complex features
    MaxPooling2D((2, 2)), 
    Conv2D(128, (3, 3), activation='relu'), # 128 filters for even more complex features
    MaxPooling2D((2, 2)),
    Flatten(), 
    Dropout(0.5), # prevent overfitting by randomly dropping 50% of the neurons during training
    Dense(128, activation='relu'), # 128 neurons in this fully connected layer
    Dense(1, activation='sigmoid') # 1 neuron for binary classification
])

c:\Repositories\computer-vision\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
""" model compilation
    Adam optimizer is an efficient optimization algorithm that adjusts the learning rate during training.
    The output layer uses 'sigmoid' activation function suitable for binary classification tasks.
    Binary crossentropy is the loss function used for binary classification tasks."""

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

![Output](assets/Output_1.png)

Output gave 3 columns: Layer (type), Output Shape and Param #. 
1. **Layer (type)** - is a type of neural network. This represents the specific mathematical operation performed on the data at this step.

    - **conv2d**: These are the "filters" scanning the X-ray for patterns like edges or cloudy areas.

    - **max_pooling2d**: This simplifies the image, reducing the resolution to keep only the most important features.

    - **dense**: This is the "brain" part of the model that takes all the patterns found and decides if they look like Pneumonia.

2. **Output Shape** - This tells you the dimensions of the data after it passes through the layer. E.g. In the image, (None, 148, 148, 32) means:

    - **None**: This is a placeholder for the Batch Size (how many images you process at once).

    - **148, 148**: The new height and width of the image (it gets smaller after convolutions).

    - **32**: The number of Filters (features) the layer is looking for.

3. **Param #** - This is the number of "weights" or "learned connections" the model is adjusting during training. 
- (First row) Param # (896): 
    - Filter size is a standard 3x3 kernel: **3 * 3 = 9**
    - Since X-rays are usually treated as RGB in these models, there are 3 channels (Red, Green, Blue): **9 * 3 = 27**)** 
    - Each filter has **1** bias weight: **27 + 1 = 28**
    - Number of Filters: This layer has **32 filters**.
    

        $$
        \text{Parameters = (filter\_height * filter\_width * input\_channels + 1) * number\_of\_filters}
        $$

        $$
        \text{28 (weights per filter) * 32 (filters) = 896 total parameters}
        $$

- (Second row) Output Shape --> (None, 72, 72, 32) - the size is reduced by half.
- Param # (0) --> Unlike a convolutional layer, which has filters (weights) that the model must adjust to learn patterns, Pooling simply follows a rule. It looks at a small window of pixels (usually 2 x 2) and simply picks the largest value (Max) to pass to the next layer. Because the rule "pick the highest number" never changes, there is nothing for the model to "train" or store in its memory for this specific layer, that is why it is '0'.

- (Third row) Param # (18,496):
    - Filter size is a standard 3x3 kernel: **3 * 3 = 9**
    - Since the previous layer had **32 filters**, so this layer receives 32 channels of data: **9 * 32 = 288**
    - Each filter has **1** bias weight: **288 + 1 = 289**
    - Number of Filters: This layer has **64 filters**.

        $$
        \text{289 (weights per filter) * 64 (filters) = 18,496 total parameters}
        $$

- Notice the first dense layer has ``4,735,104`` parameters! This is where most of the "learning" happens.

- Total parameters (18.42 MB) tells you the "memory size" of your model.


In [13]:
''' Training the model:
    steps_per_epoch: number of batches to process before declaring one epoch finished.
    epochs: number of times the model will go through the entire training dataset.
    validation_data: data on which to evaluate the loss and any model metrics at the end of each epoch.
    validation_steps: number of batches to process from the validation data at the end of each epoch.
'''
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10, # number of times the model will go through the entire training dataset
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size)

Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 63s 474ms/step - accuracy: 0.9239 - loss: 0.2061 - val_accuracy: 0.0000e+00 - val_loss: 1.2994
Epoch 2/10
  1/130 ━━━━━━━━━━━━━━━━━━━━ 26s 209ms/step - accuracy: 0.9688 - loss: 0.0557

c:\Repositories\computer-vision\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


130/130 ━━━━━━━━━━━━━━━━━━━━ 19s 142ms/step - accuracy: 0.9688 - loss: 0.0557 - val_accuracy: 0.0000e+00 - val_loss: 1.2985
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 57s 442ms/step - accuracy: 0.9539 - loss: 0.1057 - val_accuracy: 0.5244 - val_loss: 1.1249
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 19s 146ms/step - accuracy: 0.9688 - loss: 0.1655 - val_accuracy: 0.5322 - val_loss: 1.0951
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 56s 433ms/step - accuracy: 0.9626 - loss: 0.0977 - val_accuracy: 0.5977 - val_loss: 0.9987
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 1.0000 - loss: 0.0744 - val_accuracy: 0.5918 - val_loss: 1.0015
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 59s 458ms/step - accuracy: 0.9744 - loss: 0.0829 - val_accuracy: 0.7373 - val_loss: 0.9032
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 19s 149ms/step - accuracy: 0.9375 - loss: 0.1971 - val_accuracy: 0.8203 - val_loss: 0.7919
Epoch 9/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 60s 462ms/step - accuracy: 0.9751 - loss: 0.0766 -

### Testing the model on the same data

In [14]:
test_dir = 'data/chest_xray/test'
test_filepaths = []
test_labels = []

In [15]:
for category in categories:
    folder = os.path.join(test_dir, category)
    for fname in os.listdir(folder):
        test_filepaths.append(f"{category}/{fname}")
        test_labels.append(category)

In [16]:
test_df = pd.DataFrame({'Filename': test_filepaths, 'Label': test_labels})

In [17]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
# Loading training data from dataframe
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='Filename',
    y_col='Label',
    batch_size=32, # processing 32 images at a time
    shuffle=False, # shuffling data before the training
    class_mode='binary', # binary classification (NORMAL vs PNEUMONIA)
    target_size=(150, 150) # resizing images to 150x150 pixels
)

Found 624 validated image filenames belonging to 2 classes.


### Prediction step

In [19]:
pred_probs = model.predict(test_generator)
preds = (pred_probs > 0.5).astype(int).flatten()
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
print(classification_report(true_labels, preds, target_names=class_labels))
print(confusion_matrix(true_labels, preds))

20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step
              precision    recall  f1-score   support

      NORMAL       1.00      0.12      0.21       234
   PNEUMONIA       0.65      1.00      0.79       390

    accuracy                           0.67       624
   macro avg       0.83      0.56      0.50       624
weighted avg       0.78      0.67      0.57       624

[[ 28 206]
 [  0 390]]
